In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/Karrthik-Arya/Moodify-Learning/master/Assignment-3/problem%201/train.csv")
val_data = pd.read_csv("https://raw.githubusercontent.com/Karrthik-Arya/Moodify-Learning/master/Assignment-3/problem%201/val.csv")
data.head()

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx,letter
0,4,7,5,5,4,7,6,7,5,6,4,7,3,7,3,7,N
1,3,9,4,7,2,7,6,14,5,8,13,7,3,9,0,8,U
2,3,3,4,4,1,7,7,4,4,7,6,8,3,8,4,8,X
3,4,5,5,4,5,8,8,4,4,7,7,8,5,10,9,10,S
4,6,9,8,7,5,4,9,6,7,7,10,11,3,9,1,8,U


In [3]:
x_train = data.iloc[:,:-1].values.astype('float32')
x_val = val_data.iloc[:,:-1].values.astype('float32')
y_train = data.iloc[:,-1]
y_val = val_data.iloc[:,-1]
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)
def standardize(x): 
    return (x-mean_px)/std_px
x_val = standardize(x_val)
x_train = standardize(x_train)

x_train

array([[-0.6633796 ,  0.3679242 , -0.31961167, ...,  0.3679242 ,
        -1.0071476 ,  0.3679242 ],
       [-1.0071476 ,  1.0554601 , -0.6633796 , ...,  1.0554601 ,
        -2.0384514 ,  0.71169215],
       [-1.0071476 , -1.0071476 , -0.6633796 , ...,  0.71169215,
        -0.6633796 ,  0.71169215],
       ...,
       [-1.0071476 , -0.6633796 ,  0.02415627, ...,  0.3679242 ,
        -1.0071476 ,  1.0554601 ],
       [-1.0071476 ,  0.71169215, -0.6633796 , ...,  0.71169215,
        -1.6946834 ,  0.3679242 ],
       [-1.6946834 , -1.0071476 , -1.3509154 , ...,  0.3679242 ,
        -2.0384514 ,  0.3679242 ]], dtype=float32)

In [4]:
unique,inverse = np.unique(y_train,return_inverse=True)
y_train = np.eye(unique.shape[0])[inverse]

In [5]:
val_unique,val_inverse = np.unique(y_val,return_inverse=True)
y_val_ohe = np.eye(val_unique.shape[0])[val_inverse]

In [6]:
np.random.seed(42)
n1,n2,n3 = 16,20,26
w1 = np.random.randn(20,16)
b1 = np.zeros(20)
w1 = w1*math.sqrt(2.0/n1)
w2 = np.random.randn(26,20)
b2 =np.zeros(26)
w2 = w2*math.sqrt(2.0/n2)

In [7]:
def relu(x):
  return np.maximum(0,x)
def relu_der(x):
  if x<=0:
    return 0
  else:
    return 1
batch_size = 130
lr = 0.95
for k in range(20):
  loss = 0
  for y in range(len(y_train)//batch_size):
    deltaw2 = np.zeros((batch_size,26,20))
    deltab2 = np.zeros((batch_size,26))
    deltaw1 = np.zeros((batch_size,20,16))
    deltab1 = np.zeros((batch_size,20))
    temp_loss =0
    for x in range(batch_size):
      x1 = x_train[x+y*batch_size]
      x2i = w1.dot(x1) + b1
      x2 = relu(x2i)
      x3 = w2.dot(x2) + b2
      x4 = np.zeros(26)
      for i in range(26):
        x4[i] = math.exp(x3[i])/np.sum(np.exp(x3))
      ic = np.where(y_train[(x+y*batch_size)%len(y_train)]==1)[0][0]
      deltax2 = np.zeros(20)
      temp_loss+= -math.log(x4[ic])
      for i in range(26):
        if i == ic:
          delta = x4[i]-1
        else:
          delta = x4[i]
        deltab2[x][i] = delta
        for j in range(20):
          deltaw2[x][i][j] = x2[j]*delta 
          deltax2[j] += delta*w2[i][j]
      for i in range(20):
        delta1 = relu_der(x2i[i])
        deltab1[x][i] = deltax2[i]*delta1 
        for j in range(16):
          deltaw1[x][i][j] = deltax2[i]*x1[j]*delta1
    deltaw1 = np.mean(deltaw1, axis=0)
    deltab1 = np.mean(deltab1, axis =0)
    deltaw2 = np.mean(deltaw2, axis = 0)
    deltab2 = np.mean(deltab2, axis =0)
    w1 += -deltaw1*lr
    b1 += -deltab1*lr
    w2 += -deltaw2*lr
    b2 += -deltab2*lr
    temp_loss /= batch_size
    loss += temp_loss
  loss = loss/len(y_train)*batch_size
  y_pred = []
  val_loss = 0
  for i in range(len(y_val)):
    ix = np.where(y_val_ohe[i] == 1)[0][0]
    x1 = x_val[i]
    x2i = w1.dot(x1) + b1
    x2 = relu(x2i)
    x3 = w2.dot(x2) + b2
    x4 = np.zeros(26)
    for j in range(26):
      x4[j] = math.exp(x3[j])/np.sum(np.exp(x3))
    val_loss += -math.log(x4[ix])
    y_pred.append(unique[np.where(x4 == np.amax(x4))[0][0]])
  accuracy = np.mean(y_pred == y_val)
  val_loss = val_loss/len(y_val)
  print("train_loss = "+str(loss))
  print("val_loss = " + str(val_loss))
  print("val_accuracy = "+str(accuracy))
  
  

train_loss = 1.6751013130546226
val_loss = 1.0849092425569367
val_accuracy = 0.6777142857142857
train_loss = 1.0241381554812614
val_loss = 0.8982531316514196
val_accuracy = 0.7422857142857143
train_loss = 0.8876766046672471
val_loss = 0.8015781104481872
val_accuracy = 0.7668571428571429
train_loss = 0.8071723706283952
val_loss = 0.7433737985354244
val_accuracy = 0.7874285714285715
train_loss = 0.7529280944411462
val_loss = 0.6995692947250427
val_accuracy = 0.7988571428571428
train_loss = 0.7080032973864492
val_loss = 0.6673500314363333
val_accuracy = 0.8082857142857143
train_loss = 0.6721096333283568
val_loss = 0.6398945796778885
val_accuracy = 0.8131428571428572
train_loss = 0.6439996878467236
val_loss = 0.6199899538891549
val_accuracy = 0.816
train_loss = 0.6200071541186343
val_loss = 0.601082955202625
val_accuracy = 0.8234285714285714
train_loss = 0.6008140247748003
val_loss = 0.5894086295390772
val_accuracy = 0.8257142857142857
train_loss = 0.5866462032311613
val_loss = 0.579721391

In [17]:
test_data = pd.read_csv('https://raw.githubusercontent.com/Karrthik-Arya/Moodify-Learning/master/Assignment-3/problem%201/test.csv')
x_test = test_data.iloc[:,:].values.astype('float32')
x_test = standardize(x_test)

In [18]:
y_pred = []
for i in range(len(x_test)):
  x1 = x_test[i]
  x2i = w1.dot(x1) + b1
  x2 = relu(x2i)
  x3 = w2.dot(x2) + b2
  x4 = np.zeros(26)
  for i in range(26):
    x4[i] = math.exp(x3[i])/np.sum(np.exp(x3))
  y_pred.append(unique[np.where(x4 == np.amax(x4))[0][0]])

In [19]:
submissions=pd.DataFrame({"ID": list(range(1,len(y_pred)+1)),
                         "Label": y_pred})
submissions.to_csv("test_predictions.csv", index = False)